In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
import os
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal


In [2]:
load_dotenv()

True

In [3]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=os.getenv("GOOGLE_API_KEY")
)

In [4]:
prompt = PromptTemplate(
    template = "Tell me 7 facts about {topic}",
    input_variables=['topic']
)

parser = StrOutputParser()

In [5]:
# Simple chain
chain = prompt | model | parser

chain.invoke('Diabetic Retinopathy')

"Okay, here are 7 facts about Diabetic Retinopathy:\n\n1.  **It's a complication of diabetes:** Diabetic retinopathy is caused by high blood sugar levels damaging the blood vessels in the retina, the light-sensitive tissue at the back of the eye.  It's a direct result of diabetes and its impact on the circulatory system.\n\n2.  **Often has no early symptoms:** In the early stages, diabetic retinopathy may not cause any noticeable vision problems. People may not realize they have it until the condition is advanced.  This is why regular eye exams are so crucial for people with diabetes.\n\n3.  **It can lead to vision loss and blindness:** If left untreated, diabetic retinopathy can progress and cause significant vision loss, including blindness.  The damaged blood vessels can leak fluid and blood, or new, abnormal blood vessels can grow on the surface of the retina (proliferative diabetic retinopathy), which can lead to scarring and retinal detachment.\n\n4.  **There are different stages

In [6]:
prompt2 = PromptTemplate(
    template = "Tell the most interesting out of all facts {facts}",
    input_variables=['facts']
)

In [7]:
# Sequential Chain
chain = prompt | model | parser | prompt2 | model | parser

chain.invoke("Puri in Odisha")

"Out of those facts, the most interesting is the **Rath Yatra Festival**.\n\nHere's why:\n\n*   **Scale and Spectacle:** The sheer scale of the Rath Yatra, attracting millions of devotees, is mind-boggling. It's one of the largest religious gatherings in the world.\n*   **Cultural Significance:** The festival encapsulates the deep religious and cultural significance of Lord Jagannath and the unique traditions of Odisha.\n*   **Visual Impact:** The image of the massive, elaborately decorated chariots being pulled through the streets is incredibly striking and memorable.\n*   **Unique Ritual:** The act of pulling the chariots is considered a sacred duty and a way to connect with the divine.\n\nWhile the other facts are interesting, the Rath Yatra is the most captivating because of its grandeur, cultural importance, and the powerful images it evokes."

In [9]:
prompt1 = PromptTemplate(
    template = "Give the Batting stats of the {player} in all formats of Cricket",
    input_variables = ['player']
)

In [10]:
prompt2 = PromptTemplate(
    template = "Give the Bowling stats of the {player} in all formats of Cricket",
    input_variables = ['player']
)

In [11]:
prompt3 = PromptTemplate(
    template = "Merge the bowling stats -> {bowling} and batting stats -> {batting} and give in a tabular format",
    input_variables = ['bowling','batting']
)

In [12]:
parser = StrOutputParser()

In [14]:
# Parallel Chains
chain = RunnableParallel(
        {
                'bowling' : prompt2 | model | parser,
                'batting' : prompt1 | model | parser,
        }
)

In [15]:
sec_chain = prompt3 | model | parser 

In [16]:
final_chain = chain | sec_chain

In [18]:
result = chain.invoke('Rohit Sharma')

In [21]:
print(result['bowling'])

Rohit Sharma is primarily known for his batting prowess. He is a part-time bowler, and his bowling statistics are not extensive. Here's a summary of his bowling stats across all formats:

**Test Matches**

*   **Matches:** 49
*   **Innings Bowled:** 16
*   **Balls Bowled:** 428
*   **Runs Conceded:** 297
*   **Wickets:** 0
*   **Best Bowling:** 0/0
*   **Average:** N/A
*   **Economy:** 4.15
*   **Strike Rate:** N/A
*   **4 Wickets in an Innings:** 0
*   **5 Wickets in an Innings:** 0
*   **10 Wickets in a Match:** 0

**One Day Internationals (ODIs)**

*   **Matches:** 262
*   **Innings Bowled:** 44
*   **Balls Bowled:** 955
*   **Runs Conceded:** 765
*   **Wickets:** 8
*   **Best Bowling:** 2/27
*   **Average:** 95.62
*   **Economy:** 4.80
*   **Strike Rate:** 119.3
*   **4 Wickets in an Innings:** 0
*   **5 Wickets in an Innings:** 0

**Twenty20 Internationals (T20Is)**

*   **Matches:** 157
*   **Innings Bowled:** 12
*   **Balls Bowled:** 132
*   **Runs Conceded:** 163
*   **Wickets:

In [20]:
print(result['batting'])

Okay, here's a summary of Rohit Sharma's batting statistics in all three major formats of international cricket (Tests, ODIs, and T20Is), as well as his IPL stats:

**Rohit Sharma Batting Stats (As of November 5, 2023)**

*   **Tests**

    *   Matches: 54
    *   Innings: 94
    *   Not Outs: 6
    *   Runs: 3762
    *   Highest Score: 212
    *   Average: 45.9
    *   Strike Rate: 50.83
    *   100s: 10
    *   50s: 16
    *   4s: 426
    *   6s: 77

*   **ODIs (One Day Internationals)**

    *   Matches: 257
    *   Innings: 249
    *   Not Outs: 41
    *   Runs: 10534
    *   Highest Score: 264
    *   Average: 49.69
    *   Strike Rate: 90.02
    *   100s: 31
    *   50s: 55
    *   4s: 961
    *   6s: 301

*   **T20Is (Twenty20 Internationals)**

    *   Matches: 148
    *   Innings: 140
    *   Not Outs: 16
    *   Runs: 3853
    *   Highest Score: 118
    *   Average: 30.25
    *   Strike Rate: 139.24
    *   100s: 4
    *   50s: 29
    *   4s: 347
    *   6s: 182

*   **IPL (I

In [23]:
class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

In [24]:
parser2 = PydanticOutputParser(pydantic_object=Feedback)

In [25]:
prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

In [26]:
classifier_chain = prompt1 | model | parser2

In [32]:
prompt2 = PromptTemplate(
    template='Write a single appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

In [33]:
prompt3 = PromptTemplate(
    template='Write a single appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)


In [35]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

In [36]:
chain = classifier_chain | branch_chain

In [39]:
print(chain.invoke({'feedback': 'This is a wonderful phone'}))

Thank you for your positive feedback!


In [41]:
print(chain.invoke({'feedback': 'This is a horrible phone'}))

"I'm sorry to hear you had a negative experience. Could you please tell me more about what happened so I can understand how we can improve?"
